# Reading Notes Thrase.jl

In [5]:
using DifferentialEquations

In [3]:
# bdry_vec_mod!()
# F matrices are associated with imposing boundary conditions
# HfI_FT matrices are associated with computing traction

# G1 = -(Iz ⊗ ex0T) * Sx0 
# G2 = +(Iz ⊗ exNT) * SxN 
# G3 = -(ez0T ⊗ Ix) * Sz0
# G4 = +(ezNT ⊗ Ix) * SzN

# F1 = G1' - ((τ1 * H1) ⊗ ex0)
# F2 = G2' - ((τ2 * H2) ⊗ exN)
# F3 = G3' - (ez0 ⊗ (τ3 * H3))
# F4 = G4' - (ezN ⊗ (τ4 * H4))


# HfI_F1T = H1I * G1 - (τ1 ⊗ ex0')
# HfI_F2T = H2I * G2 - (τ2 ⊗ exN')
# HfI_F3T = H3I * G3 - (ez0' ⊗ τ3)
# HfI_F4T = H4I * G4 - (ezN' ⊗ τ4)

# g[:] .= 0

# # FACE 1 (Dirichlet):
# vf = bc_Dirichlet(1, 0, z)
# g[:] -= F[1] * vf

# # FACE 2 (Dirichlet):
# vf = bc_Dirichlet(2, Lx, z)
# g[:] -= F[2] * vf

# # FACE 3 (Neumann):
# gN = bc_Neumann(3, x, 0, 0, -1)
# vf = gN  ./ diag(τ[3])
# g[:] -= F[3] * vf

# # FACE 4 (Neumann):
# gN = bc_Neumann(4, x, Lz, 0, 1)
# vf = gN  ./ diag(τ[4])
# g[:] -= F[4] * vf 

In [ ]:
cb_fun = SavingCallback(
    (ψδ, t, i) -> write_to_file(pth, ψδ, t, i, z, flt_loc, flt_loc_indices,station_strings, station_indices, odeparam, "BP1_", 0.1 * year_seconds), 
    SavedValues(Float64, Float64)
)


In [1]:
function write_to_file(pth, ψδ, t, i, zf, flt_loc, flt_loc_indices, station_strings, station_indices, p, base_name="", tdump=100)
    
    # i: integrator
    path_to_slip = pth * "slip.dat"
    Vmax = 0.0
  
    if isdefined(i,:fsallast) 
      δNp = p.δNp
      Nz = p.N
      dψV = i.fsallast
      dψ = @view dψV[1:δNp]
      V = @view dψV[δNp .+ (1:Nz+1)]
      Vmax = maximum(abs.(extrema(V)))
      δ = @view ψδ[δNp .+ (1:Nz+1)]
      ψ = @view ψδ[1:δNp]
      τf = p.τf
    
   
      θ = (p.RSDc * exp.((ψ .- p.RSf0) ./ p.RSb)) / p.RSV0  # Invert ψ for θ.
    
      if mod(ctr[], p.save_stride_fields) == 0 || t == (sim_years ./ 31556926)
        vv = Array{Float64}(undef, 1, 2+length(flt_loc))
        vv[1] = t
        vv[2] = log10(Vmax)
        vv[3:end] = δ[flt_loc_indices]
        open(path_to_slip, "a") do io
          writedlm(io, vv)
        end
  
        for i = 1:length(station_indices)
          ww = Array{Float64}(undef, 1, 5)
          ww[1] = t
          ww[2] = δ[station_indices[i]]
          ww[3] = log10(V[station_indices[i]])
          ww[4] = τf[station_indices[i]]
          ww[5] = log10(θ[station_indices[i]])
  
          XXX = pth * "fltst_strk"*station_strings[i]*".txt"
          open(XXX, "a") do io
              writedlm(io, ww)
          end
        end
        
      end
    
      global ctr[] += 1
    end
  
    Vmax
  end

write_to_file (generic function with 3 methods)

In [1]:
function create_text_files(pth, flt_loc, flt_loc_indices, stations, station_strings, station_indices, t, RSVinit, δ, τz0, θ)


    path_to_slip = pth * "slip.dat"
    # slip.dat is a file that stores time, max(V) and slip at all the stations:
    open(path_to_slip, "w") do io
      write(io,"0.0 0.0 ")
        for i in 1:length(flt_loc)
          write(io,"$(flt_loc[i]) ")
        end
          write(io,"\n")
      end
    
    #write out initial data into devol.txt:
    vv = Array{Float64}(undef, 1, 2+length(flt_loc))
      vv[1] = t
      vv[2] = log10(RSVinit)
      vv[3:end] = δ[flt_loc_indices]
      open(path_to_slip, "a") do io
          writedlm(io, vv)
      end
  
    # write out initial data into station files:
  
    # fltst_dpXXX.txt is a file that stores time and time-series of slip, log10(slip_rate), 
    # shear_stress and log10(state) at depth of z = XXX km, where XXX is each of the fault station depths.
    # First we write out initial data into each fltst_dpXXX.txt:
  
    for n = 1:length(station_strings)
      XXX = pth * "fltst_strk"*station_strings[n]*".txt"
      ww = Array{Float64}(undef, 1, 5)
      ww[1] = t
      ww[2] = δ[station_indices[n]]
      ww[3] = log10(RSVinit)
      ww[4] = τz0
      ww[5] = log10(θ[station_indices[n]])  # 
      open(XXX, "w") do io
        write(io, "# problem=SEAS Benchmark BP1-QD\n")  # 
        write(io, "# code=Thrase\n")
        write(io, "# modeler=B. A. Erickson\n")
        write(io, "# date=2023/01/09\n")
        write(io, "# element size=xx m\n")
        write(io, "# location=on fault, z = "*string(parse(Int64, station_strings[n])/10)*" km\n")
        write(io, "# Lz = 40 km\n")
        write(io, "t slip slip_rate shear_stress state\n")
  
        writedlm(io, ww)
      end
    end
  
  end

create_text_files (generic function with 1 method)